In [ ]:
#*****************************************************************************************************************************************        
# Prophage annotation : 
#*****************************************************************************************************************************************        
from os import system, listdir, chdir, mkdir
from os.path import isdir
import os
import random                                   
path_klebsiella="/home/conchae/prediction_depolymerase_tropism"
path_phageboost="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/phageboost/phageboost_prediction"

good_strains=open(f"{path_klebsiella}/panacota_pangenome/panacota_pangenome_list.txt").read().split("\n")

for specie in os.listdir(path_klebsiella):
    if specie[0]=="k" and os.path.isdir(f"{path_klebsiella}/{specie}")== True:
        for strain in random.sample(os.listdir(f"{path_klebsiella}/{specie}/refseq/bacteria"), len(os.listdir(f"{path_klebsiella}/{specie}/refseq/bacteria"))):
            if strain in good_strains :
                path_fna=f"{path_klebsiella}/{specie}/refseq/bacteria/{strain}/prokka_annotation_all/{strain}.fna"
                path_prophage=f"{path_phageboost}/{strain}"
                try :
                    mkdir(path_prophage)
                except FileExistsError :
                    print("The output for phageboost already exists for some reason. We shall continue")
                if len(os.listdir(f"{path_prophage}")) == 0:
                    system(f"PhageBoost -f {path_fna} -o {path_prophage}  --threads 4")
                    with open(f"{path_prophage}/process_done","w") as outfile:
                        outfile.write("This strain has been studied")
                    
                    
# Get the prediction scores
# Writting some info files ...
from os import system, listdir, chdir, mkdir
from os.path import isdir
import os

path_phageboost="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/phageboost"

with open(f"{path_phageboost}/score_distribution.phageboost.csv","w") as outfile:
    for strain in os.listdir(f"{path_phageboost}/phageboost_prediction") :
        if len(os.listdir(f"{path_phageboost}/phageboost_prediction/{strain}")) > 2 :
            for file in os.listdir(f"{path_phageboost}/phageboost_prediction/{strain}"):
                if file[0:6]=="phages":
                    info_file=open(f"{path_phageboost}/phageboost_prediction/{strain}/{file}").read().split("\n")[2:]
                    for index_info, info in enumerate(info_file):
                        if info :
                            score=info.split("\t")[5]
                            outfile.write(f"{strain},{score}\n")


# *******************************************************************************************************************************************
# The fastANI command :
# *******************************************************************************************************************************************

import os
import pandas as pd
from tqdm import tqdm

path_phageboot_info="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_info"
path_phageboost_pred="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_prediction"
path_fastANI_2="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
path_ktype="/home/conchae/prediction_depolymerase_tropism"


    # But first, the strain_ktype dictionary :
strain_ktype={}
good_strain=open(f"{path_ktype}/results_kleborate_count.tsv").read().split("\n")
for index_strain, info in enumerate(good_strain):
    if info:
        strain=info.split("\t")[0].strip()
        ktype=info.split("\t")[2].strip()
        strain_ktype[strain]=ktype

with open(f"{path_phageboot_info}/results_phageboost.70.20102022.tsv","w") as outfile1 :
    outfile1.write(f"Prophage_name\tProphage_length\tN_genes\tScore\tK_type\n")
    for strain in tqdm(os.listdir(path_phageboost_pred)):
        # Opening the resume file of phageboost prediction :
        for file in os.listdir(f"{path_phageboost_pred}/{strain}"):
            if file[0:6]=="phages":
                try :
                    resume= pd.read_csv(f"{path_phageboost_pred}/{strain}/{file}", skiprows=1, sep="\t")
                except Exception as e:
                    print(f"Seems like there is no prophage for ")
                #Scanning the file for phage with a score > 0.70
                for index_info, info in resume.iterrows():
                    if float(info["score"])>= 0.70 :
                        # Getting the prophage info :
                        prophage_id= info["attributes"].split("phage_id=")[1]
                        prophage_len= int(info["start"]) -int(info["end"])
                        n_genes= info["attributes"].split("n_genes=")[1].split(";")[0]
                        for file2 in os.listdir(f"{path_phageboost_pred}/{strain}"):
                            if file2.count(prophage_id)>0:
                                seq=open(f"{path_phageboost_pred}/{strain}/{file2}").read().split("\n")[1]
                                if os.path.isfile(f"{path_fastANI_2}/{strain}__{prophage_id}.fasta")==False:
                                    with open(f"{path_fastANI_2}/{strain}__{prophage_id}.fasta","w") as outfile :
                                        outfile.write(f">{strain}__{prophage_id}\n{seq}")
                        outfile1.write(f"{strain}__{prophage_id}\t{str(prophage_len)}\t{n_genes}\t{info['score']}\t{strain_ktype[strain]}\n")


# Step 2 :
# Writting the path file :

with open(f"{path_phageboot_info}/fastANI_list.20102022.tsv","w") as outfile :
    for file in tqdm(os.listdir(path_fastANI_2)):
        outfile.write(f"{path_fastANI_2}/{file}\n")
        
        
#*****************************************************************************************************************************************    
# Step 3 :        
# fatANI commands : 

#!/bin/bash
#BATCH --job-name=fatANI_phb
#SBATCH --partition=medium 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=40
#SBATCH --mem=200gb 
#SBATCH --time=4-00:00:00 
#SBATCH --output=fatANI_phb%j.log 

module restore la_base
source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate fastani

fastANI  --ql /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_info/fastANI_list.20102022.tsv --rl /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_info/fastANI_list.20102022.tsv -o /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_out_20102022  --matrix  -t 40

In [ ]:
# *******************************************************************************************************************************************
# Inspecting the fastANI outputs : Approach 2 (20/04/2023)

#First round inspection : Get the pairs of prophages with a ANI score>0.99 and coverage > 80%

# Output format. In all above use cases, OUTPUT_FILE will contain tab delimited row(s) with query genome, reference genome, 
# ANI value, count of bidirectional fragment mappings, and total query fragments. 
# Alignment fraction (wrt. the query genome) is simply the ratio of mappings and total fragments. (https://github.com/ParBLiSS/FastANI)
# *******************************************************************************************************************************************

import os
import pandas as pd
from tqdm import tqdm


path_fastani="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_ktype="/home/conchae/prediction_depolymerase_tropism"

# Generating the dico with the k_type info for each strain
strain_ktype={}
good_strain=open(f"{path_ktype}/results_kleborate_count.tsv").read().split("\n")
for index_strain, info in enumerate(good_strain):
    if info:
        strain=info.split("\t")[0].strip()
        ktype=info.split("\t")[2].strip()
        strain_ktype[strain]=ktype
    
fastani_names = ["Query","Reference_genome","ANI","fragments","total_fragments"]
fastani_df = pd.read_csv(f"{path_fastani}/fastANI_out_20102022",sep="\t", names = fastani_names , nrows = 1000)
fastani_df = fastani_df[fastani_df["ANI"] >= 90]

families = []
fastani_dict = fastani_df.to_dict('records')
for row in tqdm(fastani_dict) :
    if float(row["ANI"]) >=99 and float(row["fragments"])/float(row["total_fragments"])>=0.80:
        l_query = len(open(f"{row['Query']}").read().split("\n")[1]) 
        l_refer = len(open(f"{row['Reference_genome']}").read().split("\n")[1])
        case_1 = (l_query > l_refer and l_query /l_refer >= 0.8)
        case_2 = (l_refer > l_query and l_refer /l_query >= 0.8)
        #if l_query /l_refer >= 0.8 and l_query /l_refer <= 1.25 : 
        if case_1 or case_2 :
        #if l_query /l_refer >= 0.8 and l_query /l_refer <= 1.25 : 
            prophage_1 = row["Query"].split("/")[-1]
            prophage_2 = row["Reference_genome"].split("/")[-1]
            pair = {prophage_1, prophage_2}
            for cluster in families :
                if cluster.isdisjoint(pair) == False :
                    cluster.update(pair)
                    break
            else :
                families.append(pair)


with open(f"{path_fastani}/clusters_99_80.info.2004.v2.tsv",'w') as outfile :
    with open(f"{path_fastani}/clusters_99_80.2004.v2.tsv",'w') as outfile_cluster :
        outfile.write("Family_index\tMember\n")
        outfile_cluster.write("Family_index\tMembers\n")
        for index_c, cluster in enumerate(families) :
            outfile_cluster.write(f"{index_c}\t")
            cluster_c_l = []
            for member in cluster :
                outfile.write(f"family {index_c}\t{member}\n")
                cluster_c_l.append(member)
            outfile_cluster.write(",".join(cluster_c_l))
            outfile_cluster.write("\n")

# *******************************************************************************************************************************************                    
#!/bin/bash
#BATCH --job-name=post_ANI2_
#SBATCH --qos=short 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=10
#SBATCH --mem=10gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=post_ANI2_%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate ScaleAP

python3 /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/script_files/py_files/cluster.I.2004.py
# *******************************************************************************************************************************************              
# Check the integrety of the DF :
import os
import pandas as pd
from tqdm import tqdm


path_fastani="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_phages = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
path_ktype="/home/conchae/prediction_depolymerase_tropism"

families = pd.read_csv(f"{path_fastani}/clusters_99_80.2004.v2.tsv", header = 0, sep='\t')
families_set = [set(fam.split(",")) for fam in families["Members"]]

n_iteration = 7
clean_families = []
tmp_families = families_set.copy()
# fill up the tmp clusters
for n in range(n_iteration):
    for index_set, cluster in tqdm(enumerate(tmp_families)) :
        for index_2, cluster_2 in enumerate(families_set):
            if cluster.isdisjoint(cluster_2) == False :
                cluster.update(cluster_2)
                continue
            else :
                continue

# Gather the clean clusters :
for index, cluster in enumerate(tmp_families) :
    if cluster not in clean_families :
        clean_families.append(cluster)
# *******************************************************************************************************************************************************************
# Write the final files :
with open(f"{path_fastani}/clusters_99_80.clean.2004.v2.tsv","w") as outfile :
    outfile.write("Family_index\tMembers\n")
    phages = set()
    L = len(clean_families)
    loners = []
    for index_f, family in tqdm(enumerate(clean_families_2)) :
        cluster_list = ",".join(list(family))
        outfile.write(f"Family_{index_f}\t{cluster_list}\n")
    for phage in tqdm(os.listdir(path_phages)):
        for index_f, family in enumerate(clean_families_2):
            if phage in family :
                break
        else :
            loners.append(phage)
    for index, phage in enumerate(loners) :        
        outfile.write(f"Loner_{str(L+index)}\t{phage}\n")    

# *******************************************************************************************************************************************************************
# Check the integrity of the files :
cluster = pd.read_csv(f"{path_fastani}/clusters_99_80.clean.v2.tsv", header = 0, sep="\t")
phages = []
cluster_dict = cluster.to_dict("records")

for row in tqdm(cluster_dict) :
    for member in row["Members"].split(",") :
        phages.append(member)
        
loners_df = cluster[cluster["Family_index"]=="Loner"]
fammmm_df = cluster[cluster["Family_index"]!="Loner"]

cluster = pd.read_csv(f"{path_fastani}/clusters_99_80.clean.2004.v2.tsv", header = 0, sep="\t")


with open(f"{path_fastani}/clusters_99_80.extra_clean.2004.v2.tsv","w") as outfile :
    outfile.write(f"prophage_id\tprophage\n")
    for index,row in tqdm(cluster.iterrows()) :
        for member in row["Members"].split(",") :
            outfile.write(f"prophage_{index}\t{member}\n")
            
            
# Fix the families
path_fastani="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_phages = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
path_ktype="/home/conchae/prediction_depolymerase_tropism"

families = pd.read_csv(f"{path_fastani}/clusters_99_80.tsv", header = 0, sep='\t')
families_set = [set(fam.split(",")) for fam in families["Members"]]

n_iteration = 10
clean_families = []
tmp_families = families_set.copy()
# fill up the tmp clusters
for n in range(n_iteration):
    for index_set, cluster in tqdm(enumerate(tmp_families)) :
        for index_2, cluster_2 in enumerate(families_set):
            if cluster.isdisjoint(cluster_2) == False :
                cluster.update(cluster_2)
                continue
            else :
                continue
    print(f"Iteration number {n}")

# Gather the clean clusters :
for index, cluster in enumerate(tmp_families) :
    if cluster not in clean_families :
        clean_families.append(cluster)
        
        
        
clean_families = []
for index_set, cluster in tqdm(enumerate(families_set)) :
    clean_cluster = cluster.copy()
    #print(clean_cluster)
    for index_2, cluster_2 in enumerate(families_set):
        if clean_cluster.isdisjoint(cluster_2) == False :
            clean_cluster.update(cluster_2)
            continue
        else :
            continue
    #print(clean_cluster)
    if clean_cluster not in clean_families :
        clean_families.append(clean_cluster)
        

# Repeat the iteration : 
clean_families_2 = []
for index_set, cluster in tqdm(enumerate(clean_families)) :
    clean_cluster = cluster.copy()
    for index_2, cluster_2 in enumerate(clean_families):
        if clean_cluster.isdisjoint(cluster_2) == False :
            clean_cluster.update(cluster_2)
            continue
        else :
            continue
    if clean_cluster not in clean_families_2 :
        clean_families_2.append(clean_cluster)

# *******************************************************************************************************************************************************************
n_iteration = 10
clean_families = []
for n in range(n_iteration):
    tmp_families = []
    for index_set, cluster in tqdm(enumerate(clean_families)) :
        clean_cluster = cluster.copy()
        #print(clean_cluster)
        for index_2, cluster_2 in enumerate(families_set):
            if clean_cluster.isdisjoint(cluster_2) == False :
                clean_cluster.update(cluster_2)
                continue
            else :
                continue
        #print(clean_cluster)
        if clean_cluster not in clean_families :
            clean_families.append(clean_cluster)
    
    

# *******************************************************************************************************************************************************************
with open(f"{path_fastani}/clusters_99_80.clean.tsv","w") as outfile :
    outfile.write("Family_index\tMembers\n")
    phages = set()
    L = len(clean_families)
    loners = []
    for index_f, family in tqdm(enumerate(clean_families_2)) :
        cluster_list = ",".join(list(family))
        outfile.write(f"Family_{index_f}\t{cluster_list}\n")
    for phage in tqdm(os.listdir(path_phages)):
        for index_f, family in enumerate(clean_families_2):
            if phage in family :
                break
        else :
            loners.append(phage)
    for index, phage in enumerate(loners) :        
        outfile.write(f"Loner_{str(L+index)}\t{phage}\n")
